In [ ]:
import pandas as pd
from pyprojroot import here

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data_path = here() /'data' /'dunnhumby - Breakfast at the Frat.xlsx'

In [ ]:
# Get a list of all the sheet names 
xls = pd.ExcelFile(data_path)
xls.sheet_names

In [ ]:
# Read the data
glossary = pd.read_excel(data_path,sheet_name=xls.sheet_names[0],header=3)

dh_store_lookup = pd.read_excel(data_path,sheet_name=xls.sheet_names[1],header=1)

dh_products_lookup = pd.read_excel(data_path,sheet_name=xls.sheet_names[2],header=1)

dh_transaction_data = pd.read_excel(data_path,sheet_name=xls.sheet_names[3],header=1)
dh_transaction_data.head(4)

In [ ]:
# Merge transaction data with products 
data = pd.merge(dh_transaction_data,dh_products_lookup,
                left_on=['UPC'],
                right_on=['UPC'],
                how='left')

In [ ]:
# Merge transaction data & products with store lookup information 
data_merge = pd.merge(data,dh_store_lookup,
                left_on=['STORE_NUM'],
                right_on=['STORE_ID'],
                how='left')

In [ ]:
data_merge['units per visit'] = data_merge['UNITS']/data_merge['VISITS']
data_merge.head(3)

In [ ]:
tmp = data_merge[data_merge['DESCRIPTION'] == 'ACT KIDS BBLGUM FLUOR RNS']
tmp.head(5)

In [ ]:
price_products = data_merge.groupby(['DESCRIPTION'])['UNITS'].sum()
price_products

In [ ]:
price_range_products = data_merge.groupby(['DESCRIPTION'])['PRICE'].aggregate(['min','max'])
price_range_products

In [ ]:
tmp = price_range_products.merge(price_products,how='left',on='DESCRIPTION')
tmp